In [1]:
import stanza
from collections import Counter 
from nltk.util import ngrams
from nltk.corpus import stopwords


In [2]:
#load russian language model for stanza
stanza.download('ru')

2020-11-08 20:17:34 INFO: Downloading default packages for language: ru (Russian)...
2020-11-08 20:17:37 INFO: File exists: /home/philipp/stanza_resources/ru/default.zip.
2020-11-08 20:17:47 INFO: Finished downloading models and saved to /home/philipp/stanza_resources.


In [3]:
#create stanza pipeline for tokenization, pos labels and lemmatization
nlp = stanza.Pipeline(lang="ru", processors='tokenize,pos,lemma', )

2020-11-08 20:17:47 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

2020-11-08 20:17:48 INFO: Use device: gpu
2020-11-08 20:17:48 INFO: Loading: tokenize
2020-11-08 20:17:49 INFO: Loading: pos
2020-11-08 20:17:50 INFO: Loading: lemma
2020-11-08 20:17:50 INFO: Done loading processors!


In [4]:
# load from json
import pandas as pd
from tqdm import tqdm
infile_name = "all_hp"
df = pd.read_json(f"/home/philipp/projects/text-analysis/data/{infile_name}.json")
df["len"] = df.text.map(lambda x: len(x))
df = df.where(df.len > 0).dropna().sort_values(by=["book", "chapter"]).reset_index(drop=True)
#df = df.where(df.book == "Гарри Поттер и Тайная Комната").dropna().reset_index(drop=True)
docs = [nlp(text) for text in tqdm(df.text)]

100%|██████████| 327/327 [09:01<00:00,  1.66s/it]


In [5]:
# load from plaintext
infile_name = "hp1-1"
doc = [nlp(open(f"/home/philipp/projects/text-analysis/data/{infile_name}.txt").read())]

In [33]:
n = 2
top_ngrams = 10000
lemmatize = False
remove_stopwords = False
filter_pos = ["NUM", "PUNCT", "PROPN"]
lemmatize_pos = ["NOUN", "ADJ", "VERB", "DET", "PRON"]
additional_punctuation = "«»"

# remove all numerals, punctuation and proper nouns from list 
# extract lemmatized word for nouns, adjectives and verbs
out_str = ""
lemmatize_pos = [] if not lemmatize else lemmatize_pos
lemmas = [word for doc in docs for sent in doc.sentences for word in sent.words if word.upos not in filter_pos]
total_word_count = len(lemmas)
print(f"{total_word_count}\ttotal words (without {filter_pos}) \n")
lemmas = [word.lemma if word.upos in lemmatize_pos else word.text for word in lemmas]
lemmas = [lemma.lower() for lemma in lemmas if lemma not in additional_punctuation]

# get ngrams and filter out individual words
russian_stopwords = stopwords.words("russian") if remove_stopwords else []

bi_grams = list(ngrams(lemmas, n)) 
counter = Counter(bi_grams)
wc = counter.most_common(top_ngrams)

cum = 0
i=0

header = "rank\ttotal\tcum\tword\n" if n == 1 else "rank\ttotal\tword\n"
out_str+=header
for ngram, freq in [ngram for ngram in wc]:
    cum += freq/total_word_count
    i+=1
    if not any([True for i in range(len(ngram)) if ngram[i] in russian_stopwords]):
        words = " ".join(ngram)
        row = f"{i}\t{freq}\t{cum*100:.2f}%\t{words}" if n == 1 else f"{i}\t{freq}\t{words}"
        out_str+=row+"\n"
print(out_str)

846259	total words (without ['NUM', 'PUNCT', 'PROPN']) 

rank	total	word
1	1031	у него
2	933	и не
3	896	что он
4	705	я не
5	686	он не
6	678	на него
7	542	не было
8	541	что это
9	535	и в
10	509	ничего не
11	489	и он
12	473	сказал он
13	467	с ним
14	457	потому что
15	450	то что
16	443	не мог
17	435	что ты
18	428	что я
19	425	так что
20	421	как будто
21	415	и все
22	410	и с
23	407	к нему
24	395	у меня
25	383	о том
26	378	том что
27	378	об этом
28	364	так и
29	360	он был
30	355	что не
31	352	он и
32	348	тут же
33	346	если бы
34	342	все еще
35	342	волшебную палочку
36	333	может быть
37	316	никогда не
38	313	это не
39	308	у нее
40	306	рядом с
41	304	что они
42	303	в этом
43	302	ты не
44	300	только что
45	298	глядя на
46	297	что вы
47	296	не знаю
48	294	еще не
49	285	у нас
50	284	как и
51	279	что в
52	278	но не
53	277	в сторону
54	274	что у
55	272	никто не
56	271	как он
57	270	знал что
58	270	что она
59	265	в том
60	261	его в
61	261	и на
62	259	за ним
63	259	когда они
64	256	у тебя
65	255	и т

In [32]:
# save the output
infile_name = "hp_all"
out_dir = "/home/philipp/projects/text-analysis/freq_lists/"

removed_stop = "rms" if remove_stopwords else ""
lemmatized = "lm" if lemmatize else ""
removed_other = "".join(filter_pos)
outfile_name = f"{infile_name}_{n}gram_{top_ngrams}_{removed_stop}{lemmatized}_{removed_other}"
out_path = f'{out_dir}{outfile_name}.tsv'
with open(out_path, 'w') as the_file:
    the_file.write(out_str)
print(f"Saved to {out_path}.")

Saved to /home/philipp/projects/text-analysis/freq_lists/hp_all_1gram_10000_lm_NUMPUNCTPROPN.tsv.
